<a href="https://colab.research.google.com/github/Joshua1030/APS360_Team17/blob/main/hrd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
list = [['coffee', 'soda', 'tea'], 
        ['pizza', 'hamburger', 'hotdog'], 
        ['cake', 'ice cream']]

abc = list
abc[0][1] = 'aaa'
list

[['coffee', 'aaa', 'tea'],
 ['pizza', 'hamburger', 'hotdog'],
 ['cake', 'ice cream']]

In [ ]:
from copy import deepcopy
from heapq import heappush, heappop
import time
import argparse
import sys

#====================================================================================

char_goal = '1'
char_single = '2'

class Piece:
    """
    This represents a piece on the Hua Rong Dao puzzle.
    """

    def __init__(self, is_goal, is_single, coord_x, coord_y, orientation):
        """
        :param is_goal: True if the piece is the goal piece and False otherwise.
        :type is_goal: bool
        :param is_single: True if this piece is a 1x1 piece and False otherwise.
        :type is_single: bool
        :param coord_x: The x coordinate of the top left corner of the piece.
        :type coord_x: int
        :param coord_y: The y coordinate of the top left corner of the piece.
        :type coord_y: int
        :param orientation: The orientation of the piece (one of 'h' or 'v') 
            if the piece is a 1x2 piece. Otherwise, this is None
        :type orientation: str
        """

        self.is_goal = is_goal
        self.is_single = is_single
        self.coord_x = coord_x
        self.coord_y = coord_y
        self.orientation = orientation

    def __repr__(self):
        return '{} {} {} {} {}'.format(self.is_goal, self.is_single, \
            self.coord_x, self.coord_y, self.orientation)

class Board:
    """
    Board class for setting up the playing board.
    """

    def __init__(self, pieces):
        """
        :param pieces: The list of Pieces
        :type pieces: List[Piece]
        """

        self.width = 4
        self.height = 5
        #list[pieces]
        self.pieces = pieces

        # self.grid is a 2-d (size * size) array automatically generated
        # using the information on the pieces when a board is being created.
        # A grid contains the symbol for representing the pieces on the board.
        self.grid = []
        self.__construct_grid()


    def construct_grid(self):
        """
        Called in __init__ to set up a 2-d grid based on the piece location information.

        """
        self.grid = []
        for i in range(self.height):
            line = []
            for j in range(self.width):
                line.append('.')
            self.grid.append(line)

        for piece in self.pieces:
            if piece.is_goal:
                self.grid[piece.coord_y][piece.coord_x] = char_goal
                self.grid[piece.coord_y][piece.coord_x + 1] = char_goal
                self.grid[piece.coord_y + 1][piece.coord_x] = char_goal
                self.grid[piece.coord_y + 1][piece.coord_x + 1] = char_goal
            elif piece.is_single:
                self.grid[piece.coord_y][piece.coord_x] = char_single
            else:
                if piece.orientation == 'h':
                    self.grid[piece.coord_y][piece.coord_x] = '<'
                    self.grid[piece.coord_y][piece.coord_x + 1] = '>'
                elif piece.orientation == 'v':
                    self.grid[piece.coord_y][piece.coord_x] = '^'
                    self.grid[piece.coord_y + 1][piece.coord_x] = 'v'

    def __construct_grid(self):
        """
        Called in __init__ to set up a 2-d grid based on the piece location information.

        """

        for i in range(self.height):
            line = []
            for j in range(self.width):
                line.append('.')
            self.grid.append(line)

        for piece in self.pieces:
            if piece.is_goal:
                self.grid[piece.coord_y][piece.coord_x] = char_goal
                self.grid[piece.coord_y][piece.coord_x + 1] = char_goal
                self.grid[piece.coord_y + 1][piece.coord_x] = char_goal
                self.grid[piece.coord_y + 1][piece.coord_x + 1] = char_goal
            elif piece.is_single:
                self.grid[piece.coord_y][piece.coord_x] = char_single
            else:
                if piece.orientation == 'h':
                    self.grid[piece.coord_y][piece.coord_x] = '<'
                    self.grid[piece.coord_y][piece.coord_x + 1] = '>'
                elif piece.orientation == 'v':
                    self.grid[piece.coord_y][piece.coord_x] = '^'
                    self.grid[piece.coord_y + 1][piece.coord_x] = 'v'

    def display(self):
        """
        Print out the current board.

        """
        for i, line in enumerate(self.grid):
            for ch in line:
                print(ch, end='')
            print()
        

class State:
    """
    State class wrapping a Board with some extra current state information.
    Note that State and Board are different. Board has the locations of the pieces. 
    State has a Board and some extra information that is relevant to the search: 
    heuristic function, f value, current depth and parent.
    """

    def __init__(self, board, f, depth, parent=None):
        """
        :param board: The board of the state.
        :type board: Board
        :param f: The f value of current state.
        :type f: int
        :param depth: The depth of current state in the search tree.
        :type depth: int
        :param parent: The parent of current state.
        :type parent: Optional[State]
        """
        self.board = board
        self.f = f
        self.depth = depth
        self.parent = parent
        self.id = hash(board)  # The id for breaking ties.

    def __lt__(self, other):
         return self.f < other.f


def read_from_file(filename):
    """
    Load initial board from a given file.

    :param filename: The name of the given file.
    :type filename: str
    :return: A loaded board
    :rtype: Board
    """

    puzzle_file = open(filename, "r")

    line_index = 0
    pieces = []
    g_found = False

    for line in puzzle_file:

        for x, ch in enumerate(line):

            if ch == '^': # found vertical piece
                pieces.append(Piece(False, False, x, line_index, 'v'))
            elif ch == '<': # found horizontal piece
                pieces.append(Piece(False, False, x, line_index, 'h'))
            elif ch == char_single:
                pieces.append(Piece(False, True, x, line_index, None))
            elif ch == char_goal:
                if g_found == False:
                    pieces.append(Piece(True, False, x, line_index, None))
                    g_found = True
        line_index += 1

    puzzle_file.close()

    board = Board(pieces)
    
    return board


def goalTest(curr_state: State) -> bool:
    """
    check if the current state is the goal state
    """
    curr_board = curr_state.board
    curr_board.construct_grid()
    if curr_board.grid[3][1] == char_goal and curr_board.grid[3][2] == char_goal and \
            curr_board.grid[4][1] == char_goal and curr_board.grid[4][2] == char_goal:
        return True
    else:
        return False

def backtracking(goal_state: State) -> list:
    solution = [goal_state]
    current_state = goal_state
    while current_state.parent:
        current_state = current_state.parent
        solution.insert(0, current_state)
    return solution
    

def isLegalMove(coord_y: int, coord_x: int) -> bool:
    if coord_y <= 4 and coord_y >= 0 and coord_x <= 3 and coord_x >= 0:
        return True
    else:
        return False

def isup(curr_board: Board, piece: Piece) -> bool:
    #check if this piece can be moved up

    #if the piece is ^
    if piece.orientation == 'v':
        if isLegalMove(piece.coord_y - 1, piece.coord_x):
            #check if the above space is empty
            if curr_board.grid[piece.coord_y - 1][piece.coord_x] == '.':
                return True
            return False
        else:
            return False
    #if the piece is <
    elif piece.orientation == 'h':
        if isLegalMove(piece.coord_y - 1, piece.coord_x) and \
              isLegalMove(piece.coord_y - 1, piece.coord_x + 1):
            #check if the above space is empty
            if curr_board.grid[piece.coord_y - 1][piece.coord_x] == '.' and \
                curr_board.grid[piece.coord_y - 1][piece.coord_x + 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 1 goal
    elif piece.is_goal:
        if isLegalMove(piece.coord_y - 1, piece.coord_x) and \
              isLegalMove(piece.coord_y - 1, piece.coord_x + 1):
            #check if the above space is empty
            if curr_board.grid[piece.coord_y - 1][piece.coord_x] == '.' and \
                curr_board.grid[piece.coord_y - 1][piece.coord_x + 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 2 single
    elif piece.is_single:
        if isLegalMove(piece.coord_y - 1, piece.coord_x):
            #check if the above space is empty
            if curr_board.grid[piece.coord_y - 1][piece.coord_x] == '.':
                return True
            return False
        else:
            return False

def isdown(curr_board: Board, piece: Piece) -> bool:
    #if the piece is ^
    if piece.orientation == 'v':
        if isLegalMove(piece.coord_y + 2, piece.coord_x):
            #check if the below space is empty
            if curr_board.grid[piece.coord_y + 2][piece.coord_x] == '.':
                return True
            return False
        else:
            return False
    #if the piece is <
    elif piece.orientation == 'h':
        if isLegalMove(piece.coord_y + 1, piece.coord_x) and \
              isLegalMove(piece.coord_y + 1, piece.coord_x + 1):
            #check if the below space is empty
            if curr_board.grid[piece.coord_y + 1][piece.coord_x] == '.' and \
                curr_board.grid[piece.coord_y + 1][piece.coord_x + 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 1 goal
    elif piece.is_goal:
        if isLegalMove(piece.coord_y + 2, piece.coord_x) and \
              isLegalMove(piece.coord_y + 2, piece.coord_x + 1):
            #check if the below space is empty
            if curr_board.grid[piece.coord_y + 2][piece.coord_x] == '.' and \
                curr_board.grid[piece.coord_y + 2][piece.coord_x + 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 2 single
    elif piece.is_single:
        if isLegalMove(piece.coord_y + 1, piece.coord_x):
            #check if the below space is empty
            if curr_board.grid[piece.coord_y + 1][piece.coord_x] == '.':
                return True
            return False
            
        else:
            return False

def isleft(curr_board: Board, piece: Piece) -> bool:
    #if the piece is ^
    if piece.orientation == 'v':
        if isLegalMove(piece.coord_y, piece.coord_x - 1) and \
              isLegalMove(piece.coord_y + 1, piece.coord_x - 1):
            #check if the left space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x - 1] == '.' and \
                curr_board.grid[piece.coord_y + 1][piece.coord_x - 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is <
    elif piece.orientation == 'h':
        if isLegalMove(piece.coord_y, piece.coord_x - 1):
            #check if the left space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x - 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 1 goal
    elif piece.is_goal:
        if isLegalMove(piece.coord_y, piece.coord_x - 1) and \
              isLegalMove(piece.coord_y + 1, piece.coord_x - 1):
            #check if the left space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x - 1] == '.' and \
                curr_board.grid[piece.coord_y + 1][piece.coord_x - 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 2 single
    elif piece.is_single:
        if isLegalMove(piece.coord_y, piece.coord_x - 1):
            #check if the left space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x - 1] == '.':
                return True
            return False
        else:
            return False
        
def isright(curr_board: Board, piece: Piece) -> bool:
    #if the piece is ^
    if piece.orientation == 'v':
        if isLegalMove(piece.coord_y, piece.coord_x + 1) and \
              isLegalMove(piece.coord_y + 1, piece.coord_x + 1):
            #check if the right space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x + 1] == '.' and \
                curr_board.grid[piece.coord_y + 1][piece.coord_x + 1] == '.':
                return True
            return False
        else:
            return False
    #if the piece is <
    elif piece.orientation == 'h':
        if isLegalMove(piece.coord_y, piece.coord_x + 2):
            #check if the right space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x + 2] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 1 goal
    elif piece.is_goal:
        if isLegalMove(piece.coord_y, piece.coord_x + 2) and \
              isLegalMove(piece.coord_y + 1, piece.coord_x + 2):
            #check if the right space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x + 2] == '.' and \
                curr_board.grid[piece.coord_y + 1][piece.coord_x + 2] == '.':
                return True
            return False
        else:
            return False
    #if the piece is 2 single
    elif piece.is_single:
        if isLegalMove(piece.coord_y, piece.coord_x + 1):
            #check if the left space is empty
            if curr_board.grid[piece.coord_y][piece.coord_x + 1] == '.':
                return True
            return False
        else:
            return False


def modify_piece_list(new_piece_list: list, previous_one_piece: Piece, \
                        new_coord_y: int, new_coord_x: int):
    #modify state.board.pieces[index].coordinate
    piece_index = 0
    for i, piece in enumerate(new_piece_list):
        if piece.coord_x == previous_one_piece.coord_x and \
            piece.coord_y == previous_one_piece.coord_y:
            piece_index = i
            break
    
    new_piece_list[piece_index].coord_x = new_coord_x
    new_piece_list[piece_index].coord_y = new_coord_y

#modify new state
def move_up(new_state: State, piece: Piece):
    new_state.f += 1
    new_state.depth += 1
    new_board = new_state.board
    new_piece_lsit = new_board.pieces
    modify_piece_list(new_piece_lsit, piece, piece.coord_y-1, piece.coord_x)
    #update the board.grid
    new_board.construct_grid()
        
def move_down(new_state: State, piece: Piece):
    new_state.f += 1
    new_state.depth += 1
    new_board = new_state.board
    new_piece_lsit = new_board.pieces
    modify_piece_list(new_piece_lsit, piece, piece.coord_y+1, piece.coord_x)
    #update the board.grid
    new_board.construct_grid()

def move_left(new_state: State, piece: Piece):
    new_state.f += 1
    new_state.depth += 1
    new_board = new_state.board
    new_piece_lsit = new_board.pieces
    modify_piece_list(new_piece_lsit, piece, piece.coord_y, piece.coord_x-1)
    #update the board.grid
    new_board.construct_grid()

def move_right(new_state: State, piece: Piece):
    new_state.f += 1
    new_state.depth += 1
    new_board = new_state.board
    new_piece_lsit = new_board.pieces
    modify_piece_list(new_piece_lsit, piece, piece.coord_y, piece.coord_x+1)
    #update the board.grid
    new_board.construct_grid()


def successorGenerator(curr_state: State) -> list:
    successors = []
    #if you modify curr_board, also modify curr_state.board. 
    curr_board = curr_state.board
    #curr_board.construct_grid()

    for piece in curr_board.pieces:
        if isup(curr_board, piece):

            new_board = deepcopy(curr_state.board)
            new_state = State(new_board, curr_state.f, curr_state.depth, curr_state)

            move_up(new_state, piece)
            successors.append(new_state)

        if isdown(curr_board, piece):

            new_board = deepcopy(curr_state.board)
            new_state = State(new_board, curr_state.f, curr_state.depth, curr_state)

            move_down(new_state, piece)
            successors.append(new_state)

        if isleft(curr_board, piece):

            new_board = deepcopy(curr_state.board)
            new_state = State(new_board, curr_state.f, curr_state.depth, curr_state)

            move_left(new_state, piece)
            successors.append(new_state)

        if isright(curr_board, piece):

            new_board = deepcopy(curr_state.board)
            new_state = State(new_board, curr_state.f, curr_state.depth, curr_state)
            
            move_right(new_state, piece)
            successors.append(new_state)
    return successors

def manhattanDistance(child_state: State) -> int:
    #(3,1) goal coordination
    for piece in child_state.board.pieces:
        if piece.is_goal:
            y = piece.coord_y
            x = piece.coord_x

    return abs(1-x)+abs(3-y)

def dfs(initial_state: State) -> list:
    """
    Removing the newest state from the list(frontier) to find the goal state
    crate a output_file - contains each steps from initial state to goal state

    args:
        initial_state - initial configuration of the board  with f, depth, and parent.
    return:
        list[State]
    """

    """
    corner case
    if goalTest(initial_state):
        write_outputfile(initial_state)
    """
    frontier = []
    exploredSet = dict()
    frontier.append(initial_state)
    #frontier is not empty do
    while frontier:
        curr = frontier.pop()
        #check if find the goal state
        if goalTest(curr):
            print("steps for dfs: ", curr.depth)
            return backtracking(curr)
        #multi-path prunning
        result_grid_string = ' '.join([str(item) for item in curr.board.grid])
        if result_grid_string not in exploredSet:
            exploredSet[result_grid_string] = 'existed'
            #find the curr's successors and add to frontier
            for state in successorGenerator(curr):
                frontier.append(state)
    #no solution
    print("No solution\r")


def AstarSearch(initial_state: State) -> list:
    #using priority queue as the frontier
    #f = g + h
    #g = state.depth
    #h = manhattan distance
    frontier = []
    exploredSet = dict()
    heappush(frontier, (0, initial_state))
    while frontier:
        value, curr_state = heappop(frontier)
        if goalTest(curr_state):
            print("steps for A*: ", curr_state.depth)
            return backtracking(curr_state)
        #multi-path prunning
        result_grid_string = ' '.join([str(item) for item in curr_state.board.grid])
        if result_grid_string not in exploredSet:
            exploredSet[result_grid_string] = 1
            #find the curr's successors and add to frontier
            for state in successorGenerator(curr_state):
                g = state.depth
                h = manhattanDistance(state)
                state.f = g + h
                f = state.f
                heappush(frontier, (f, state))

    #no solution
    print("No solution\r")
        

def test_func(face):
    """
    Load initial board from a given file.

    :param filename: The name of the given file.
    :type filename: str
    :return: A loaded board
    :rtype: Board
    """


    line_index = 0
    pieces = []
    g_found = False

    for line in face:

        for x, ch in enumerate(line):

            if ch == '^': # found vertical piece
                pieces.append(Piece(False, False, x, line_index, 'v'))
            elif ch == '<': # found horizontal piece
                pieces.append(Piece(False, False, x, line_index, 'h'))
            elif ch == char_single:
                pieces.append(Piece(False, True, x, line_index, None))
            elif ch == char_goal:
                if g_found == False:
                    pieces.append(Piece(True, False, x, line_index, None))
                    g_found = True
        line_index += 1

    board = Board(pieces)
    
    return board

def write_file(State_list, filename):
    with open(filename, 'w') as f:
            #every state
            for state in State_list:
                #state.board.display()
                for row in state.board.grid:
                    f.write("".join(str(x) for x in row) + "\n")
                # write each item on a new line
                f.write("\n")
            print('Done')
    f.close()

if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--inputfile",
        type=str,
        required=True,
        help="The input file that contains the puzzle."
    )
    parser.add_argument(
        "--outputfile",
        type=str,
        required=True,
        help="The output file that contains the solution."
    )
    parser.add_argument(
        "--algo",
        type=str,
        required=True,
        choices=['astar', 'dfs'],
        help="The searching algorithm."
    )
    args = parser.parse_args()

    # read the board from the file
    board = read_from_file(args.inputfile)
    initial_state = State(board, 0, 0, None)
    #create grid[]
    #initial_state.board.construct_grid()
    if args.algo == 'astar':
    # perform actions for astar algorithm
        sol_list = AstarSearch(initial_state)
        write_file(sol_list, args.outputfile)
    elif args.algo == 'dfs':
    # perform actions for dfs algorithm
        sol_list = dfs(initial_state)
        #sol_list=[State]
        write_file(sol_list, args.outputfile)

        
    




